In [1]:

!pip install -q datasets torch torchvision

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
import torch.cuda.amp as amp
from torchvision import transforms
from datasets import load_dataset
import os
import datetime
import matplotlib.pyplot as plt
from google.colab import drive

# Data loading
try:
    drive.mount('/content/drive')
    checkpoint_dir = '/content/drive/MyDrive/DCGAN_Checkpoints'
    os.makedirs(checkpoint_dir, exist_ok=True)
    print(f"Checkpoints will be saved to: {checkpoint_dir}")
except:
    print("Drive mounting skipped or failed. Checkpoints will be local only.")
    checkpoint_dir = 'checkpoints'
    os.makedirs(checkpoint_dir, exist_ok=True)

#Dataset preparation

# Load dataset from HuggingFace
ds = load_dataset("zh-plus/tiny-imagenet", split="train")

# Define transformations (Resize to 64x64, Normalize to [-1, 1])
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# Wrapper class to handle HuggingFace format
class dataset_wrap(torch.utils.data.Dataset):
    def __init__(self, ds, transform=None):
        self.ds = ds
        self.transform = transform

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, indx):
        item = self.ds[indx]
        img = item['image'].convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img

# Create the full dataset wrapper
full_dataset = dataset_wrap(ds, transform=transform)

# create a subset (first 20,000 images) for SPEED
subset_indices = range(20000)
small_dataset = Subset(full_dataset, subset_indices)

# Create DataLoader
data_loader = DataLoader(
    dataset=small_dataset,
    batch_size=128,   # Optimized batch size
    shuffle=True,
    num_workers=2,    # Parallel loading
    pin_memory=True,  # Fast CPU->GPU transfer
    drop_last=True
)

# Optimized models for fast results

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self, x):
        resi = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += resi
        return F.relu(out)

# Discrimnator

class Discriminator(nn.Module):
    def __init__(self, in_channel=3, base_dim=128):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channel, base_dim, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(base_dim),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.res1 = ResidualBlock(base_dim)
        self.conv2 = nn.Sequential(
            nn.Conv2d(base_dim, base_dim * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(base_dim * 2),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.res2 = ResidualBlock(base_dim * 2)
        self.conv3 = nn.Sequential(
            nn.Conv2d(base_dim * 2, base_dim * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(base_dim * 4),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.res3 = ResidualBlock(base_dim * 4)
        self.con4 = nn.Sequential(
            nn.Conv2d(base_dim * 4, base_dim * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(base_dim * 4),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.res4 = ResidualBlock(base_dim * 4)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(base_dim * 4, 1)

        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.res1(x)
        x = self.conv2(x)
        x = self.res2(x)
        x = self.conv3(x)
        x = self.res3(x)
        x = self.con4(x)
        x = self.res4(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Generator

class Generator(nn.Module):
    def __init__(self, n_noise=100, out_channel=3, base_dim=128):
        super(Generator, self).__init__()
        self.init_dim = 4
        self.base_dim = base_dim * 4
        self.fc = nn.Sequential(
            nn.Linear(n_noise, self.base_dim * (self.init_dim ** 2)),
            nn.ReLU(inplace=True)
        )
        self.g1 = nn.Sequential(
            nn.ConvTranspose2d(self.base_dim, self.base_dim // 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.base_dim // 2),
            nn.ReLU(inplace=True)
        )
        self.res1 = ResidualBlock(self.base_dim // 2)
        self.g2 = nn.Sequential(
            nn.ConvTranspose2d(self.base_dim // 2, self.base_dim // 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.base_dim // 4),
            nn.ReLU(inplace=True)
        )
        self.res2 = ResidualBlock(self.base_dim // 4)
        self.g3 = nn.Sequential(
            nn.ConvTranspose2d(self.base_dim // 4, self.base_dim // 8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.base_dim // 8),
            nn.ReLU(inplace=True)
        )
        self.res3 = ResidualBlock(self.base_dim // 8)
        self.g4 = nn.Sequential(
            nn.ConvTranspose2d(self.base_dim // 8, out_channel, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.fc(x)
        x = x.view(x.size(0), self.base_dim, self.init_dim, self.init_dim)
        x = self.g1(x)
        x = self.res1(x)
        x = self.g2(x)
        x = self.res2(x)
        x = self.g3(x)
        x = self.res3(x)
        x = self.g4(x)
        return x

# Initializing

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Running on device: {DEVICE}")

D = Discriminator().to(DEVICE)
G = Generator(n_noise=100).to(DEVICE)

criterion = nn.BCEWithLogitsLoss() # Changed to BCEWithLogitsLoss
D_opt = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
G_opt = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Helper function for sampling
IMAGE_DIM = (64, 64, 3)
def get_sample_image(G, n_noise):
    z = torch.randn(10, n_noise).to(DEVICE)
    y_hat = G(z).view(10, IMAGE_DIM[2], IMAGE_DIM[0], IMAGE_DIM[1]).permute(0, 2, 3, 1)
    result = (y_hat.detach().cpu().numpy() + 1) / 2.
    return result

# Training loop

max_epoch = 10
n_noise = 100
n_critic = 1
scaler = amp.GradScaler()



print("Starting Optimized Training Loop...")

step = 0
for epoch in range(max_epoch):
    for idx, images in enumerate(data_loader):
        x = images.to(DEVICE, non_blocking=True)
        b_size = x.size(0)


        real_label = torch.full((b_size, 1), 1.0, device=DEVICE, dtype=torch.float)
        fake_label = torch.full((b_size, 1), 0.0, device=DEVICE, dtype=torch.float)

        # Training Discriminator
        D.zero_grad()
        with amp.autocast():
            d_out_real = D(x)
            d_loss_real = criterion(d_out_real, real_label)
        scaler.scale(d_loss_real).backward()

        z = torch.randn(b_size, n_noise, device=DEVICE)
        with amp.autocast():
            fake_images = G(z)
            d_out_fake = D(fake_images.detach())
            d_loss_fake = criterion(d_out_fake, fake_label)
        scaler.scale(d_loss_fake).backward()
        scaler.step(D_opt)

        # Training Generator
        if step % n_critic == 0:
            G.zero_grad()
            with amp.autocast():
                d_out_for_g = D(fake_images)
                g_loss = criterion(d_out_for_g, real_label)
            scaler.scale(g_loss).backward()
            scaler.step(G_opt)

        scaler.update()

        # Saving images and logs for Epochs
        if step % 200 == 0:
            d_loss = d_loss_real + d_loss_fake
            dt = datetime.datetime.now().strftime('%H:%M:%S')
            print(f'Epoch: {epoch+1}/{max_epoch}, Step: {step}, D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}, Time: {dt}')

            G.eval()
            with torch.no_grad():
                samples = get_sample_image(G, n_noise)
                if not os.path.exists('samples'):
                    os.makedirs('samples')
                plt.imsave(f'samples/resdcgan_epoch{epoch+1}_step{step:05d}.jpg', samples[0])
            G.train()

        step += 1

    # Checkpoints (if learning stops in between)
    if (epoch + 1) % 2 == 0:
        save_path = f'{checkpoint_dir}/checkpoint_epoch_{epoch+1}.pth'
        torch.save({
            'epoch': epoch,
            'G_state_dict': G.state_dict(),
            'D_state_dict': D.state_dict(),
        }, save_path)
        print(f" Checkpoint saved: {save_path}")

print("Training finished. Check 'samples' folder for results.")

Mounted at /content/drive
Checkpoints will be saved to: /content/drive/MyDrive/DCGAN_Checkpoints


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-1359597a978bc4(…):   0%|          | 0.00/146M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Running on device: cuda:0
Starting Optimized Training Loop...


/tmp/ipython-input-921451600.py:209: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
/tmp/ipython-input-921451600.py:227: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/tmp/ipython-input-921451600.py:233: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/tmp/ipython-input-921451600.py:243: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Epoch: 1/10, Step: 0, D Loss: 1.4210, G Loss: 0.9506, Time: 04:41:20
Epoch: 2/10, Step: 200, D Loss: 0.3994, G Loss: 3.7304, Time: 04:42:06
 Checkpoint saved: /content/drive/MyDrive/DCGAN_Checkpoints/checkpoint_epoch_2.pth
Epoch: 3/10, Step: 400, D Loss: 0.0212, G Loss: 4.8247, Time: 04:43:03
Epoch: 4/10, Step: 600, D Loss: 0.0093, G Loss: 5.5629, Time: 04:43:52
 Checkpoint saved: /content/drive/MyDrive/DCGAN_Checkpoints/checkpoint_epoch_4.pth
Epoch: 6/10, Step: 800, D Loss: 0.0366, G Loss: 4.6513, Time: 04:44:48
 Checkpoint saved: /content/drive/MyDrive/DCGAN_Checkpoints/checkpoint_epoch_6.pth
Epoch: 7/10, Step: 1000, D Loss: 0.0097, G Loss: 5.8248, Time: 04:45:43
Epoch: 8/10, Step: 1200, D Loss: 0.0037, G Loss: 6.2481, Time: 04:46:33
 Checkpoint saved: /content/drive/MyDrive/DCGAN_Checkpoints/checkpoint_epoch_8.pth
Epoch: 9/10, Step: 1400, D Loss: 0.0054, G Loss: 6.2253, Time: 04:47:28
 Checkpoint saved: /content/drive/MyDrive/DCGAN_Checkpoints/checkpoint_epoch_10.pth
Training finish